In [1]:
import pandas as pd
import numpy as np
from modules.levelling import level_up
# disable chained assignments (bc who understands .loc properly??)
pd.options.mode.chained_assignment = None 

# Load Data:
All data sourced from https://serenesforest.net/engage/ - Thank you very much to them.

In [2]:
def load_data():
    base_stats = pd.read_csv('data/base_stats.csv').fillna(0)
    char_growth = pd.read_csv('data/char_growth.csv').fillna(0)
    class_base = pd.read_csv('data/class_base.csv').fillna(0)
    class_growth = pd.read_csv('data/class_growth.csv').fillna(0)
    class_max = pd.read_csv('data/class_max.csv').fillna(0)
    
    return base_stats, char_growth, class_base, class_growth, class_max

In [3]:
# Load databases:
base_stats, char_growth, class_base, class_growth, class_max = load_data()

# Stat growth columns:
sg_cols = char_growth.columns.tolist()[1:]
sg_cols

['HP', 'Str', 'Mag', 'Dex', 'Spd', 'Def', 'Res', 'Lck', 'Bld']

In [5]:
# Create dictionary of the stat importance for each advanced class based on allocated category:
# N.B. Only used to create my own unique rating of characters, doesn't affect anything else.
cat_dict = {'Phy': [0.2, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
            'Hyb': [0.2, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
            'Rng_P': [0.1, 1.0, 0.0, 1.0, 1.1, 1.0, 1.0, 1.0, 1.0],
            'Rng_M': [0.1, 0.0, 1.0, 1.0, 1.1, 1.0, 1.0, 1.0, 1.0],
            'None': [0.2, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
}

# Character Comparisons:

In [8]:
# Lists and dictionaries of Advanced classes:
uniq_adv = class_base[(class_base['Unique'] !=0) & (class_base['Type'] == 'Advanced')][['Name', 'Unique']]
ua_dict = dict(zip(uniq_adv.Unique, uniq_adv.Name))
non_uniq = class_base[(class_base['Unique'] ==0) & (class_base['Type'] == 'Advanced')]['Name'].tolist()
non_uniq.append('Thief')
non_uniq[0:3]

['Swordmaster', 'Hero', 'Halberdier']

In [10]:
# Character list:
char_list = base_stats.Name.tolist()
char_list.remove('Veyle')                              # Missing some of the data for Veyle, so excluded
print(len(char_list))
char_list[0:3]

35


['Alear', 'Vander', 'Clanne']

## Looping Characters and Classes:

In [11]:
int_tgt = 50                                                                          # Training all to internal level n                      
all_char_stats = []


for char in char_list:
    print("Processing %s stats.\n---------------------------------" % char)
    
    # Cycle through all Advanced Class options:
    for pro_class in non_uniq:
        base_stats, char_growth, class_base, class_growth, class_max = load_data()    # Resetting data
        # Return dataframe of stat changes throughout levelling:
        r_df = level_up(base_stats, char_growth, char, 
                        class_base, class_growth, class_max, 
                        int_tgt, promo_class=pro_class, base_tgt=10,
                        sg_cols=sg_cols, cat_dict=cat_dict)
        r_df['Promo_Class'] = pro_class
        all_char_stats.append(r_df)
    
    # Check whether a unique class is available:
    try:
        base_stats, char_growth, class_base, class_growth, class_max = load_data()    # Resetting data
        unique_class = ua_dict[char]
        r_df = level_up(base_stats, char_growth, char, 
                        class_base, class_growth, class_max, 
                        int_tgt, promo_class=unique_class, base_tgt=10,
                        sg_cols=sg_cols, cat_dict=cat_dict)
        r_df['Promo_Class'] = unique_class
        all_char_stats.append(r_df)
    except KeyError:
        pass
     

Processing Alear stats.
---------------------------------
Processing Vander stats.
---------------------------------
Processing Clanne stats.
---------------------------------
Processing Framme stats.
---------------------------------
Processing Alfred stats.
---------------------------------
Processing Etie stats.
---------------------------------
Processing Boucheron stats.
---------------------------------
Processing Celine stats.
---------------------------------
Processing Chloe stats.
---------------------------------
Processing Louis stats.
---------------------------------
Processing Yunaka stats.
---------------------------------
Processing Alcryst stats.
---------------------------------
Processing Citrinne stats.
---------------------------------
Processing Lapis stats.
---------------------------------
Processing Diamant stats.
---------------------------------
Processing Amber stats.
---------------------------------
Processing Jade stats.
---------------------------------

## Creating Dataset of Results for Plotting:

Below I created the data that has been used in the Data Visualization notebook.

In [14]:
# Concatenating all character dataframes together:
concat_df = pd.concat(all_char_stats)

# Adjusting dataframe to summarize stats by name, internal level and class:
melted_df = concat_df.melt(id_vars=["Name", "int_lev", "Class", "Promo_Class"], var_name="Metric", value_name="Value")

# Only retaining the key metrics:
melt_final = melted_df.loc[melted_df['Metric'].isin(sg_cols)].reset_index(drop=True)

In [15]:
print(melt_final.shape)
melt_final.head()

(239184, 6)


,Name,int_lev,Class,Promo_Class,Metric,Value
0,Alear,1,Dragon Child,Swordmaster,HP,22
1,Alear,2.0,Dragon Child,Swordmaster,HP,23.0
2,Alear,3.0,Dragon Child,Swordmaster,HP,24.0
3,Alear,4.0,Dragon Child,Swordmaster,HP,24.0
4,Alear,5.0,Dragon Child,Swordmaster,HP,25.0


In [16]:
melt_final.to_csv('data/char_levelled_stats.csv', index=False)

## Single Levelling Example:

If you just want to look at a single character progress through the levels you can view all the stats like the table below.

The level_up function assumes below that Anna will level to 10 and then immediately use a master seal, followed by a second seal to revert to be a Level 1 sage - hence her level below never shows as 10. The "int_lev" column tracks how many real levels the characters have trained for (including any levels they begin with, i.e. Vander). The promo_rt column is a different rating system which is controlled by the weightings specified in "cat_dict" at the top of this notebook, you can experiment with your own ratings by altering that dictionary.

In [17]:
character = 'Anna'
promotion = 'Sage'
levels = 30

level_up(base_stats, char_growth, character, 
                        class_base, class_growth, class_max, 
                        levels, promo_class=promotion, base_tgt=10,
                        sg_cols=sg_cols, cat_dict=cat_dict)

,Name,Level,Class,HP,Str,Mag,Dex,Spd,Def,Res,Lck,Bld,int_lev,game_rt,promo_rt
0,Anna,5,Axe Fighter,29,10,2,9,7,4,5,3,7,5,47.0,41.0
1,Anna,6.0,Axe Fighter,30.0,10,2,10.0,8.0,4,5,3,7,6.0,49.0,43.0
2,Anna,7.0,Axe Fighter,31.0,11.0,3.0,10.0,8.0,4,6.0,4.0,7,7.0,53.0,46.0
3,Anna,8.0,Axe Fighter,32.0,11.0,3.0,11.0,9.0,4,6.0,4.0,7,8.0,55.0,48.0
4,Anna,9.0,Axe Fighter,32.0,11.0,4.0,11.0,9.0,5.0,6.0,5.0,7,9.0,58.0,51.0
5,Anna,1,Sage,27.0,4.0,13.0,15.0,12.0,5.0,15.0,7.0,5,10.0,76.0,76.0
6,Anna,2.0,Sage,28.0,4.0,14.0,15.0,13.0,5.0,15.0,8.0,5,11.0,79.0,79.0
7,Anna,3.0,Sage,28.0,4.0,15.0,16.0,13.0,5.0,16.0,8.0,5,12.0,82.0,82.0
8,Anna,4.0,Sage,29.0,4.0,16.0,16.0,14.0,6.0,17.0,9.0,5,13.0,87.0,87.0
9,Anna,5.0,Sage,29.0,4.0,17.0,17.0,14.0,6.0,17.0,10.0,5,14.0,90.0,90.0
